In [1]:
import requests
import geopy


# Step 2: Extract all the roads within the city boundaries using the Overpass API
# use city name to get city streets from OpenStreetMap
city = "Nice"
# construct the URL for the Overpass API request
url = f"https://overpass-api.de/api/interpreter?data=[out:json];area[name={city}];(way[highway](area);>;);out;"
# send the API request
response = requests.get(url)
# save the response as a JSON file
with open("data/roads.json", "w") as file:
    file.write(response.text)
roads = response.json()


# Step 3: Divide the roads into equal fragments of 50 meters
points = []
# each road line we want to divide into equal fragments and get points
for road in roads["elements"]:
    # we want to get the coordinates of the road line
    coordinates = []
    for coordinate in road["geometry"]:
        coordinates.append((coordinate["lat"], coordinate["lon"]))

    # we want to divide the road line into equal fragments of 50 meters
    # and get the points of the fragments
    for i in range(len(coordinates) - 1):
        # we want to get the start and end coordinates of the fragment
        start = coordinates[i]
        end = coordinates[i + 1]

        # we want to get the distance between the start and end coordinates
        distance = geopy.distance.distance(start, end).meters

        # we want to divide the distance into equal fragments of 50 meters
        fragments = distance // 50

        # we want to get the latitude and longitude coordinates of the fragments
        for j in range(int(fragments)):
            # we want to get the latitude and longitude coordinates of the fragment
            latitude = start[0] + (end[0] - start[0]) * j / fragments
            longitude = start[1] + (end[1] - start[1]) * j / fragments

            # we want to save the latitude and longitude coordinates of the fragment
            points.append((latitude, longitude))



# Define the API key for accessing the Google Street View API
api_key = "YOUR_API_KEY"
# Step 4: Use the obtained coordinates to request Google Street View images
for fragment in points:
    latitude = fragment.latitude
    longitude = fragment.longitude

    # Construct the URL for the Street View API request
    url = f"https://maps.googleapis.com/maps/api/streetview?size=640x640&location={latitude},{longitude}&key={api_key}"

    # Send the API request and save the image
    response = requests.get(url)
    with open(f"street_view_{fragment.id}.jpg", "wb") as file:
        file.write(response.content)


KeyError: 'geometry'